In [4]:
from llama_stack_client import LlamaStackClient
from llama_stack_client.types import Document
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.agent import Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os

In [11]:
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

client = LlamaStackClient(
    base_url="http://localhost:8321",
    provider_data={
        "fireworks_api_key": os.environ["FIREWORKS_API_KEY"]
    }
)

urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "datasets.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]

attachments = [
    {
        "content": f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        "mime_type": "text/plain",
    }

    for i, url in enumerate(urls)
]

simple_agent = Agent(client, model=MODEL_ID, 
                     instructions="You are a helpful assistant that can answer questions about the Torchtune project.")

In [12]:
simple_session_id = simple_agent.create_session(session_name=f"simple_session_{uuid.uuid4()}")
response = simple_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": "What precision formats does torchtune support?"
            }
        ],
        documents=attachments,
        session_id=simple_session_id,
        stream=False
    )

pprint(response)

session_response = client.agents.session.retrieve(agent_id=simple_agent.agent_id, session_id=simple_session_id)
pprint(session_response)

Turn(
│   input_messages=[
│   │   UserMessage(
│   │   │   content='What precision formats does torchtune support?',
│   │   │   role='user',
│   │   │   context='.. _memory_optimization_overview_label:\n\n============================\nMemory Optimization Overview\n============================\n\n**Author**: `Salman Mohammadi <https://github.com/SalmanMohammadi>`_\n\ntorchtune comes with a host of plug-and-play memory optimization components which give you lots of flexibility\nto ``tune`` our recipes to your hardware. This page provides a brief glossary of these components and how you might use them.\nTo make things easy, we\'ve summarized these components in the following table:\n\n.. csv-table:: Memory optimization components\n   :header: "Component", "When to use?"\n   :widths: auto\n\n   ":ref:`glossary_precision`", "You\'ll usually want to leave this as its default ``bfloat16``. It uses 2 bytes per model parameter instead of 4 bytes when using ``float32``."\n   ":ref:`glossary_act_ckpt`", "Use when you\'re memory constrained and want to use a larger model, batch size or context length. Be aware that it will slow down training speed."\n   ":ref:`glossary_act_off`", "Similar to activation checkpointing, this can be used when memory constrained, but may decrease training speed. This **should** be used alongside activation checkpointing."\n   ":ref:`glossary_grad_accm`", "Helpful when memory-constrained to simulate larger batch sizes. Not compatible with optimizer in backward. Use it when you can already fit at least one sample without OOMing, but not enough of them."\n   ":ref:`glossary_low_precision_opt`", "Use when you want to reduce the size of the optimizer state. This is relevant when training large models and using optimizers with momentum, like Adam. Note that lower precision optimizers may reduce training stability/accuracy."\n   ":ref:`glossary_opt_in_bwd`", "Use it when you have large gradients and can fit a large enough batch size, since this is not compatible with ``gradient_accumulation_steps``."\n   ":ref:`glossary_cpu_offload`", "Offloads optimizer states and (optionally) gradients to CPU, and performs optimizer steps on CPU. This can be used to significantly reduce GPU memory usage at the cost of CPU RAM and training speed. Prioritize using it only if the other techniques are not enough."\n   ":ref:`glossary_lora`", "When you want to significantly reduce the number of trainable parameters, saving gradient and optimizer memory during training, and significantly speeding up training. This may reduce training accuracy"\n   ":ref:`glossary_qlora`", "When you are training a large model, since quantization will save 1.5 bytes * (# of model parameters), at the potential cost of some training speed and accuracy."\n   ":ref:`glossary_dora`", "a variant of LoRA that may improve model performance at the cost of slightly more memory."\n\n\n.. note::\n\n  In its current state, this tutorial is focused on single-device optimizations. Check in soon as we update this page\n  for the latest memory optimization features for distributed fine-tuning.\n\n.. _glossary_precision:\n\n\nModel Precision\n---------------\n\n*What\'s going on here?*\n\nWe use the term "precision" to refer to the underlying data type used to represent the model and optimizer parameters.\nWe support two data types in torchtune:\n\n.. note::\n\n  We recommend diving into Sebastian Raschka\'s `blogpost on mixed-precision techniques <https://sebastianraschka.com/blog/2023/llm-mixed-precision-copy.html>`_\n  for a deeper understanding of concepts around precision and data formats.\n\n* ``fp32``, commonly referred to as "full-precision", uses 4 bytes per model and optimizer parameter.\n* ``bfloat16``, referred to as "half-precision", uses 2 bytes per model and optimizer parameter - effectively half\n  the memory of ``fp32``, and also improves training speed. Generally, if your hardware supports training with ``bfloat16``,\n  we recommend using it - this is th

Session(
│   session_id='6910f07f-f8e0-407b-8441-60a90e7b1834',
│   session_name='simple_session_fd65775f-25c4-465b-86e9-7cee68d15130',
│   started_at=datetime.datetime(2025, 3, 22, 19, 29, 16, 862958, tzinfo=datetime.timezone.utc),
│   turns=[
│   │   Turn(
│   │   │   input_messages=[
│   │   │   │   UserMessage(
│   │   │   │   │   content='What precision formats does torchtune support?',
│   │   │   │   │   role='user',
│   │   │   │   │   context='.. _memory_optimization_overview_label:\n\n============================\nMemory Optimization Overview\n============================\n\n**Author**: `Salman Mohammadi <https://github.com/SalmanMohammadi>`_\n\ntorchtune comes with a host of plug-and-play memory optimization components which give you lots of flexibility\nto ``tune`` our recipes to your hardware. This page provides a brief glossary of these components and how you might use them.\nTo make things easy, we\'ve summarized these components in the following table:\n\n.. csv-table:: Memory optimization components\n   :header: "Component", "When to use?"\n   :widths: auto\n\n   ":ref:`glossary_precision`", "You\'ll usually want to leave this as its default ``bfloat16``. It uses 2 bytes per model parameter instead of 4 bytes when using ``float32``."\n   ":ref:`glossary_act_ckpt`", "Use when you\'re memory constrained and want to use a larger model, batch size or context length. Be aware that it will slow down training speed."\n   ":ref:`glossary_act_off`", "Similar to activation checkpointing, this can be used when memory constrained, but may decrease training speed. This **should** be used alongside activation checkpointing."\n   ":ref:`glossary_grad_accm`", "Helpful when memory-constrained to simulate larger batch sizes. Not compatible with optimizer in backward. Use it when you can already fit at least one sample without OOMing, but not enough of them."\n   ":ref:`glossary_low_precision_opt`", "Use when you want to reduce the size of the optimizer state. This is relevant when training large models and using optimizers with momentum, like Adam. Note that lower precision optimizers may reduce training stability/accuracy."\n   ":ref:`glossary_opt_in_bwd`", "Use it when you have large gradients and can fit a large enough batch size, since this is not compatible with ``gradient_accumulation_steps``."\n   ":ref:`glossary_cpu_offload`", "Offloads optimizer states and (optionally) gradients to CPU, and performs optimizer steps on CPU. This can be used to significantly reduce GPU memory usage at the cost of CPU RAM and training speed. Prioritize using it only if the other techniques are not enough."\n   ":ref:`glossary_lora`", "When you want to significantly reduce the number of trainable parameters, saving gradient and optimizer memory during training, and significantly speeding up training. This may reduce training accuracy"\n   ":ref:`glossary_qlora`", "When you are training a large model, since quantization will save 1.5 bytes * (# of model parameters), at the potential cost of some training speed and accuracy."\n   ":ref:`glossary_dora`", "a variant of LoRA that may improve model performance at the cost of slightly more memory."\n\n\n.. note::\n\n  In its current state, this tutorial is focused on single-device optimizations. Check in soon as we update this page\n  for the latest memory optimization features for distributed fine-tuning.\n\n.. _glossary_precision:\n\n\nModel Precision\n---------------\n\n*What\'s going on here?*\n\nWe use the term "precision" to refer to the underlying data type used to represent the model and optimizer parameters.\nWe support two data types in torchtune:\n\n.. note::\n\n  We recommend diving into Sebastian Raschka\'s `blogpost on mixed-precision techniques <https://sebastianraschka.com/blog/2023/llm-mixed-precision-copy.html>`_\n  for a deeper understanding of concepts around precision and data formats.\n\n* ``fp32``, commonly referred to as "full-precision", uses 4 bytes per model and optimiz